# Import necessory libraries

In [1]:
import pandas as pd
import requests
import time
import nltk
from nltk.corpus import wordnet
import pickle
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

# Data preprocess function

In [ ]:
def DataPreprocess(data):
    is_noun = lambda pos: pos[:2] == 'NN'

    def removespecial(s):
        r=list(s)
        for i in range(len(r)):
            if not(r[i].isalpha() or r[i].isdigit()):
                r[i]=' '
        return "".join(r)

    def preprocess(s):
        t=s.split()
        i=0
        while i<len(t):
            if len(t[i])==1 and t[i] in ['s','t']:
                t[i-1]+=t[i]
                t.pop(i)
            else:
                i+=1
        return " ".join(t)

    def removesingle(s):
        t=s.split()
        i=0
        while i<len(t):
            t[i]=t[i].lower()
            if len(t[i])==1 and t[i] not in ['i','a']:
                t.pop(i)
            else:
                i+=1
        return " ".join(t)

    nouns_title=[]
    nouns_content=[]

    for i in data.index:  
        print(i)
        try:
            
            s=removespecial(data['Content'][i])
            
            s=preprocess(s)
            
            s=removesingle(s)
            
            tokenized = nltk.word_tokenize(s)
            #print(tokenized)
            nouns1 = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]


            s=removespecial(data['Title'][i])
            s=preprocess(s)
            s=removesingle(s)
            tokenized = nltk.word_tokenize(s)
            nouns2 = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]

            nouns_content.append(nouns1)
            nouns_title.append(nouns2)
        except:
            pass
        
    return nouns_title,nouns_content



# Find context words function

In [ ]:
def findContextWords(nouns_title,nouns_content):
  context_words=[]
  for i in range(len(nouns_content)):
      sim=[]
      nouns=nouns_title[i]
      if len(nouns)==0:
          pass
      else:
          for j in nouns_content[i]:
              s=0
              for n in nouns:
                  syn1 = wordnet.synsets(j)
                  syn2 = wordnet.synsets(n)
                  for s1 in syn1:
                      for s2 in syn2:
                          if s1.wup_similarity(s2)!=None:
                              s+=s1.wup_similarity(s2)
              if s/len(nouns)>=1:
                  context_words.append(j)
      
  return context_words


## For V1 Data

In [ ]:
for t in ['terror','cancer','crime','health']:
    d=pd.read_csv('Train Data/NYTimes API Data/final'+t+'.csv')
    
    nouns_title,nouns_content=DataPreprocess(d)
    context_words=findContextWords(nouns_title,nouns_content)
    
    with open('Context and Ngram/V1/context_words_'+t+'.pickle', 'wb') as f:
        pickle.dump(context_words, f)
    
    print(t,'Completed')

## For V2 Data

In [ ]:
for t in ['terror','cancer','crime','health']:
    d=pd.read_csv('Train Data/NYTimes API Data/final'+t+'.csv')
    
    if t=='crime':
        d1=pd.read_csv('Train Data/Guardian API Data/train'+t+'.csv')
        d=pd.concat([d,d1[:][:1500]])
    elif t=='cancer':
        d1=pd.read_csv('Train Data/Guardian API Data/train'+t+'.csv')
        d=pd.concat([d,d1[:][:500]])
    elif t=='terror':
        d1=pd.read_csv('Train Data/Guardian API Data/train'+t+'.csv')
        d=pd.concat([d,d1[:][:500]])
    
    nouns_title,nouns_content=DataPreprocess(d)
    context_words=findContextWords(nouns_title,nouns_content)
    
    with open('Context and Ngram/V2/context_words_train'+t+'.pickle', 'wb') as f:
        pickle.dump(context_words, f)
    
    print(t,'Completed')